In [1]:
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from sklearn.metrics import f1_score
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
from sklearn.model_selection import train_test_split



pd.options.mode.chained_assignment = None

In [2]:
# memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

# import psutil
# import humanize
# import os
# import GPUtil as GPU

# GPUs = GPU.getGPUs()
# gpu = GPUs[0]
# def printm():
#     process = psutil.Process(os.getpid())
#     print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), 
#           " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
#     print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(
#         gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))


In [3]:
torch.cuda.empty_cache()
#printm()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!ls '/content/drive/My Drive/app'

'Binary Model Atelectasis.ipynb'    devkit		    test
'Binary Model Cardiomegaly.ipynb'   glove.6B.50d.txt	    text_binary.csv
'Binary Model Edema.ipynb'	    hw3-krekun.ipynb	    text.csv
'Binary Model.ipynb'		    MF_with_pytorch.ipynb   text_multilabel.csv
'Binary Model SuppDev.ipynb'	    Multiclass.ipynb	    train
 cars_test			    ratings.csv		    valid
 cars_train			    resized_data.csv


In [6]:
mimic_data = pd.read_csv("/content/drive/My Drive/app/text_binary.csv")
mimic_data.head()

,study_id,subject_id,text,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,58522792,16567081,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,58213163,16567081,b' FINAL REPOR...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,59835582,16043746,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,51487790,16456872,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,59750073,16824069,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [7]:
mimic_data = mimic_data.loc[mimic_data['No Finding'] == 0]
mimic_data.head()

,study_id,subject_id,text,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,58522792,16567081,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,58213163,16567081,b' FINAL REPOR...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,50629293,16059088,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6,50113713,16059088,b' WET READ: ___ ___ ___ 7:27 PM\n Left PICC ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
7,57620089,16059088,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
binary_data = mimic_data[['text','Cardiomegaly']]
binary_data.text = binary_data.text.str.lower()
binary_data.head()

,text,Cardiomegaly
0,"b"" final repor...",0.0
1,b' final repor...,1.0
5,"b"" final repor...",0.0
6,b' wet read: ___ ___ ___ 7:27 pm\n left picc ...,0.0
7,b' final repor...,0.0


In [9]:
binary_data.groupby('Cardiomegaly').count()

,text
Cardiomegaly,
0.0,107527
1.0,44845


In [11]:
negative = binary_data.loc[binary_data['Cardiomegaly']==0]
print(len(negative))
positive = binary_data.loc[binary_data['Cardiomegaly']==1].sample(n=len(negative),replace=True)

107527


In [12]:
binary_data = positive.append(negative,ignore_index=True).reset_index(drop=True)
#df1.append(df2, ignore_index=True)

In [13]:
len(binary_data)

215054

In [14]:
binary_data.text[0]

"b'                                 final report\\n examination:  chest (portable ap)\\n \\n indication:  ___ y/o w/unknown pmh s/p tpa and intra-arterial thrombectomy of l\\n mca occlusion.  // pna?     pna?\\n \\n impression: \\n \\n in comparison with the study of ___, the monitoring and support devices\\n are essentially unchanged. continued enlargement of the cardiac silhouette\\n with engorged pulmonary vessels consistent with increasing pulmonary venous\\n pressure.  in view of the parenchymal changes, it would be difficult to\\n unequivocally exclude superimposed pneumonia radiographically in the\\n appropriate clinical setting.\\n'"

In [15]:
y = binary_data['Cardiomegaly']
X = binary_data.drop(columns=['Cardiomegaly'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [16]:
X_train.head()

,text
0,b' final repor...
1,"b"" final repor..."
2,b' final repor...
3,b' final repor...
4,b' final repor...


In [17]:
# remove all '\\n' from the text
re_newlines = re.compile('\\\\n')
def sub_newlines(x): return re_newlines.sub('',x)

# remove all special characters from the text, keep only alphanumeric and spaces
re_letters = re.compile('[^A-Za-z0-9 ]')
def sub_letters(x): return re_letters.sub('', x)

# remove excessive spacing otherwise you end up with " " substrings
re_spaces = re.compile('\s+')
def sub_spaces(x): return re_spaces.sub(' ', x)
                
# tokenize all words.
my_tok = spacy.load('en')
def spacy_tok(x): 
    return [tok.text for tok in my_tok.tokenizer(sub_spaces
                                                 (sub_letters
                                                 (sub_newlines(x))))]

In [18]:
sub_spaces(sub_letters(sub_newlines(binary_data.text[0])))

'b final report examination chest portable ap indication yo wunknown pmh sp tpa and intraarterial thrombectomy of l mca occlusion pna pna impression in comparison with the study of the monitoring and support devices are essentially unchanged continued enlargement of the cardiac silhouette with engorged pulmonary vessels consistent with increasing pulmonary venous pressure in view of the parenchymal changes it would be difficult to unequivocally exclude superimposed pneumonia radiographically in the appropriate clinical setting'

In [19]:
#spacy_tok(binary_data.text[0])[1:]

In [20]:
binary_train = binary_data.copy()
binary_train.head()

,text,Cardiomegaly
0,b' final repor...,1.0
1,b' final repor...,1.0
2,b' final repor...,1.0
3,b' final repor...,1.0
4,b' final repor...,1.0


In [21]:
def get_counts(text):
    counts = Counter()
    for word in text:
        counts.update(spacy_tok(word)[1:])
    return counts

In [22]:
word_count = get_counts(binary_train.text)

In [23]:
len(word_count.keys())

31584

In [24]:
for word in list(word_count):
    if word_count[word] < 3:
        del word_count[word]

In [25]:
len(word_count.keys())

16580

In [26]:
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [27]:
len(words)

16582

In [28]:
def encode_sentence(text):
    enc = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in text.split()])
    return enc

In [29]:
encode_sentence(binary_train.text[0])

array([ 1,  2,  1,  1,  5,  1,  1,  1,  1,  1,  1,  1, 11,  1, 13, 14,  1,
       16, 17,  1, 19,  1,  1,  1,  1,  1,  1,  1,  1, 23, 24, 25, 26, 27,
       17,  1, 26, 28, 14, 29,  1, 31, 32,  1, 34, 35, 17, 26, 36,  1, 25,
       38, 39, 40, 41, 25, 42, 39,  1,  1, 23, 45, 17, 26, 46,  1, 48, 49,
       50, 51,  1, 53, 54, 55, 56, 57, 23,  1, 58, 59,  1])

## Dataset and data loaders

In [30]:
class Binary_Mimic(Dataset):
    def __init__(self, X, y, vocab):
        self.x = [encode_sentence(x) for x in X.text]
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y

In [31]:
b_train = Binary_Mimic(X_train, y_train, vocab2index)
b_valid = Binary_Mimic(X_val, y_val, vocab2index)

In [32]:
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (sentences, labels).
    
    Need custom collate_fn because merging sequences (including padding) is not 
    supported in default. Sequences are padded to the maximum length of mini-batch 
    sequences (dynamic padding).
    
    Args:
        data: list of tuple (sentence, label). 
            - list of word indices of variable length
            - label, 0 or 1
    Returns:
        packed_batch: (PackedSequence), see torch.nn.utils.rnn.pack_padded_sequence
        sencences: torch tensor of shape (batch_size, max_len).
        labels: torch tensor of shape (batch_size, 1).
        lengths: list; valid length for each padded sentence. 
    """
    # Sort a data list by sentences length (descending order).
    data.sort(key=lambda x: len(x[0]), reverse=True)
    sentences, labels = zip(*data)
    
    # stack labels
    labels = torch.Tensor(labels)
    
    # Merge sentences
    lengths = [len(s) for s in sentences]
   
    sents = torch.zeros(len(sentences), max(lengths)).long()
    for i, s in enumerate(sentences):
        end = lengths[i]
        sents[i, :end] = torch.Tensor(s[:end])        
    
    return sents, lengths, labels

In [33]:
# data = [b_train[0], b_train[1], b_train[2]]
# data

In [34]:
# collate_fn(data)

In [35]:
# train_loader = DataLoader(b_train, batch_size=2, shuffle=True, collate_fn=collate_fn)
# sents, lengths, labels = next(iter(train_loader))
# sents

In [36]:
# sents.shape, labels.shape

In [37]:
# lengths

## Training

In [38]:
def train_epocs(model, optimizer, train_dl, valid_dl, epochs=10):
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.cuda().long()
            y = y.cuda().float()
            y_pred = model(x, s)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, valid_dl)
        print("Epoch #%.f: train loss %.3f val loss %.3f and val accuracy %.3f" % 
              (i+1,sum_loss/total, val_loss, val_acc))
  

In [39]:
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    with torch.no_grad():
      for x, s, y in valid_dl:
          x = x.cuda().long()
          y = y.cuda().float().unsqueeze(1)
          y_hat = model(x, s)
          loss = F.binary_cross_entropy_with_logits(y_hat, y)
          y_pred = y_hat > 0
          correct += (y_pred.float() == y).float().sum()
          total += y.shape[0]
          sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

## Basic GRU Model

In [40]:
class GRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(GRUModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, lengths):
        #print(x.shape)
        x = self.embeddings(x)
        x = self.dropout(x)
        pack = pack_padded_sequence(x, lengths, batch_first=True)
        out_pack, ht = self.gru(pack)
        return self.linear(ht[-1])

In [41]:
torch.cuda.empty_cache()
#printm()

In [42]:
batch_size = 25000
train_dl = DataLoader(b_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dl = DataLoader(b_valid, batch_size=batch_size, collate_fn=collate_fn)

In [43]:
vocab_size = len(words)
print(vocab_size)
model = GRUModel(vocab_size, 50, 50).cuda()

parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

16582


In [44]:
train_epocs(model, optimizer, train_dl, valid_dl, epochs=15)

Epoch #1: train loss 0.685 val loss 0.654 and val accuracy 0.620
Epoch #2: train loss 0.625 val loss 0.583 and val accuracy 0.705
Epoch #3: train loss 0.538 val loss 0.486 and val accuracy 0.792
Epoch #4: train loss 0.453 val loss 0.398 and val accuracy 0.835
Epoch #5: train loss 0.400 val loss 0.371 and val accuracy 0.848
Epoch #6: train loss 0.358 val loss 0.314 and val accuracy 0.873
Epoch #7: train loss 0.318 val loss 0.280 and val accuracy 0.887
Epoch #8: train loss 0.283 val loss 0.249 and val accuracy 0.901
Epoch #9: train loss 0.252 val loss 0.229 and val accuracy 0.910
Epoch #10: train loss 0.234 val loss 0.215 and val accuracy 0.917
Epoch #11: train loss 0.219 val loss 0.207 and val accuracy 0.920
Epoch #12: train loss 0.207 val loss 0.195 and val accuracy 0.925
Epoch #13: train loss 0.197 val loss 0.188 and val accuracy 0.929
Epoch #14: train loss 0.192 val loss 0.186 and val accuracy 0.929
Epoch #15: train loss 0.184 val loss 0.179 and val accuracy 0.932


## Use Glove to create pre-trained embedding 

In [45]:
def loadGloveModel(gloveFile='/content/drive/My Drive/app/glove.6B.50d.txt'):
    """ Loads word vectors into a dictionary."""
    f = open(gloveFile,'r')
    word_vecs = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        word_vecs[word] = np.array([float(val) for val in splitLine[1:]])
    return word_vecs

In [46]:
word_vecs = loadGloveModel()

In [47]:
word_count = get_counts(binary_train.text)

In [48]:
print(len(word_vecs.keys()), len(word_count.keys()))

400000 31584


In [49]:
def delete_rare_words(word_vecs, word_count, min_df=4):
    """ Deletes rare words from word_count
    
    Deletes words from word_count if they are not in word_vecs
    and don't have at least min_df occurrencies in word_count.
    """
    words_delete = []
    for word in word_count:
        if word_count[word] < min_df and word not in word_vecs:
            words_delete.append(word)
    for word in words_delete: word_count.pop(word)
    return word_count

In [50]:
word_count = delete_rare_words(word_vecs, word_count, min_df=3)
print(len(word_count.keys()))

20092


In [51]:
def create_embedding_matrix(word_vecs, word_count, min_df=4, emb_size=50):
    """Creates embedding matrix from word vectors. """
    word_count = delete_rare_words(word_vecs, word_count, min_df)
    V = len(word_count.keys()) + 2
    vocab2index = {}
    W = np.zeros((V, emb_size), dtype="float32")
    vocab = ["", "UNK"]
    # adding a vector for padding
    W[0] = np.zeros(emb_size, dtype='float32')
    # adding a vector for rare words 
    W[1] = np.random.uniform(-0.25, 0.25, emb_size)
    vocab2index["UNK"] = 1
    i = 2
    for word in word_count:
        if word in word_vecs:
            W[i] = word_vecs[word]
            vocab2index[word] = i
            vocab.append(word)
            i += 1
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
            vocab2index[word] = i
            vocab.append(word)
            i += 1   
    return W, np.array(vocab), vocab2index

In [52]:
pretrained_weight, vocab, vocab2index = create_embedding_matrix(word_vecs, word_count, min_df=3)

In [53]:
len(pretrained_weight)

20094

In [54]:
emb_size = 50
V = len(pretrained_weight)
emb = nn.Embedding(V, emb_size)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0820, -0.0200,  0.0733,  ..., -0.1758, -0.1609,  0.0050],
        [-0.4124,  0.6493, -0.5585,  ...,  0.2621,  0.1045, -0.4430],
        ...,
        [-0.6107, -0.0947,  0.2727,  ...,  0.4242,  0.9317,  0.2807],
        [-0.1321, -0.3749, -0.5517,  ...,  0.9329,  0.3362, -0.2008],
        [ 0.8547,  0.1916,  0.0963,  ...,  0.1707, -0.2616,  0.5350]])

# GRU Model with Pretrained embedding layer:

In [55]:
class GRUModel_pre(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights=None) :
        super(GRUModel_pre,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if glove_weights is not None:
            self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
            self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x, lengths):
        x = self.embeddings(x)
        x = self.dropout(x)
        pack = pack_padded_sequence(x, lengths, batch_first=True)
        out_pack, ht = self.gru(pack)
        return self.linear(ht[-1])

In [56]:
torch.cuda.empty_cache()
#printm()

In [57]:
vocab_size = len(pretrained_weight)
model = GRUModel_pre(vocab_size, 50, 50, glove_weights=pretrained_weight).cuda()

parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

In [58]:
train_epocs(model, optimizer,train_dl, valid_dl, epochs=5)

Epoch #1: train loss 0.691 val loss 0.681 and val accuracy 0.566
Epoch #2: train loss 0.670 val loss 0.760 and val accuracy 0.609
Epoch #3: train loss 0.682 val loss 0.663 and val accuracy 0.638
Epoch #4: train loss 0.673 val loss 0.665 and val accuracy 0.592
Epoch #5: train loss 0.664 val loss 0.640 and val accuracy 0.638


In [59]:
model.embeddings.weight.requires_grad = True

In [60]:
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)
train_epocs(model, optimizer, train_dl, valid_dl, epochs=15)

Epoch #1: train loss 0.639 val loss 0.580 and val accuracy 0.688
Epoch #2: train loss 0.554 val loss 0.482 and val accuracy 0.776
Epoch #3: train loss 0.483 val loss 0.410 and val accuracy 0.821
Epoch #4: train loss 0.416 val loss 0.334 and val accuracy 0.861
Epoch #5: train loss 0.356 val loss 0.283 and val accuracy 0.884
Epoch #6: train loss 0.306 val loss 0.261 and val accuracy 0.897
Epoch #7: train loss 0.278 val loss 0.230 and val accuracy 0.906
Epoch #8: train loss 0.254 val loss 0.221 and val accuracy 0.913
Epoch #9: train loss 0.238 val loss 0.209 and val accuracy 0.917
Epoch #10: train loss 0.225 val loss 0.200 and val accuracy 0.921
Epoch #11: train loss 0.215 val loss 0.194 and val accuracy 0.924
Epoch #12: train loss 0.206 val loss 0.186 and val accuracy 0.926
Epoch #13: train loss 0.198 val loss 0.181 and val accuracy 0.928
Epoch #14: train loss 0.192 val loss 0.176 and val accuracy 0.931
Epoch #15: train loss 0.185 val loss 0.172 and val accuracy 0.933
